# Multimodal Machine Learning
## Ejemplo de uso

Uno de los frameworks mas utilizados en Machine Learning es Tensorflow/Keras, el cual tambien puede ser utilizado para analisis multimodal.

Este ejemplo es basado en: https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

El problema es de regresion, donde se busca una tendencia en el precio de venta de casa.

El dataset utilizado es: https://github.com/emanhamed/Houses-dataset

Este dataset incluye datos numéricos/categóricos junto con datos de imágenes para cada una de las 535 casas de ejemplo.


Primeramente se importan los paquetes necesarios:

- _pyimagesearch:_ Para el manejo del dataset
- _sklearn:_ Para el manejo de los datos de entrenamiento y prueba
- _tensorflow/keras:_ Para el manejo del modelo

In [42]:
from pyimagesearch import datasets
from pyimagesearch import models
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
import numpy as np

Se obtiene el dataset con los features de las casas y las imagenes. se define la intensidad de las imagens en tre 0 y 1:

In [43]:
print("[INFO] loading house attributes...")
df = datasets.load_house_attributes("Houses_Dataset/HousesInfo.txt")

print("[INFO] loading house images...")
images = datasets.load_house_images(df, "Houses_Dataset")
images = images / 255.0

[INFO] loading house attributes...
[INFO] loading house images...


Se hace particionamiento del set de datos en training y testing

In [32]:
print("[INFO] processing data...")
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

[INFO] processing data...


Se escala el feature "price" a su valor maximo:

In [33]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

Se procesan los features de las casas, se hace one-hot encoding en los features categóricas y se concatenan:

In [34]:
(trainAttrX, testAttrX) = datasets.process_house_attributes(df,trainAttrX, testAttrX)

Se crea el modelo de red convolucional:

In [36]:
# create the MLP and CNN models
mlp = models.create_mlp(trainAttrX.shape[1], regress=False)
cnn = models.create_cnn(64, 64, 3, regress=False)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

# compile the model using mean absolute percentage error as our loss,
# implying that we seek to minimize the absolute percentage difference
# between our price *predictions* and the *actual prices*
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

Se entrena el modelo:

In [37]:
print("[INFO] training model...")
model.fit(x=[trainAttrX, trainImagesX], y=trainY, validation_data=([testAttrX, testImagesX], testY), epochs=20, batch_size=8)

[INFO] training model...
Epoch 1/20
34/34 [==============================] - 1s 33ms/step - loss: 87.9984 - val_loss: 71.7051
Epoch 2/20
34/34 [==============================] - 1s 29ms/step - loss: 74.3483 - val_loss: 70.2611
Epoch 3/20
34/34 [==============================] - 1s 29ms/step - loss: 65.4212 - val_loss: 69.9152
Epoch 4/20
34/34 [==============================] - 1s 30ms/step - loss: 63.0343 - val_loss: 64.7282
Epoch 5/20
34/34 [==============================] - 1s 30ms/step - loss: 52.0591 - val_loss: 60.8723
Epoch 6/20
34/34 [==============================] - 1s 30ms/step - loss: 49.3257 - val_loss: 55.9489
Epoch 7/20
34/34 [==============================] - 1s 30ms/step - loss: 48.6011 - val_loss: 50.2104
Epoch 8/20
34/34 [==============================] - 1s 30ms/step - loss: 41.1185 - val_loss: 44.3261
Epoch 9/20
34/34 [==============================] - 1s 41ms/step - loss: 39.2347 - val_loss: 40.5564
Epoch 10/20
34/34 [==============================] - 1s 30ms/step 

Se hace una prediccion:

In [39]:
print("[INFO] predicting house prices...")
preds = model.predict([testAttrX, testImagesX])

[INFO] predicting house prices...


Se calcula la diferencia entre los precios de vivienda *predichos* y los precios de vivienda *reales*, luego se calcula la diferencia porcentual y la diferencia porcentual absoluta:

In [40]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

Se calcula la media y la desviación estándar de la diferencia porcentual absoluta:

In [26]:
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

Finalmente, se muestran algunas estadísticas sobre nuestro modelo:

In [41]:
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(locale.currency(df["price"].mean(), grouping=True),locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 39.76%, std: 48.06%
